### Project 02: Create a GUI Notebook Program

enter your name and date here

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [1]:
# imports
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import datetime # one module for working with dates and times
import json

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    # Initializing the main window
    def __init__(self):
            super().__init__()
            self.title('Notebook')
            self.geometry("600x400")
            self.notes = []
            # Create buttons for New Note, Save Notebook, and Open Notebook functionalities
            tk.Button(self, text="New Note", command=self.new_note).pack(side=tk.TOP, fill=tk.X)
            tk.Button(self, text="Save Notebook", command=self.save_notebook).pack(side=tk.TOP, fill=tk.X)
            tk.Button(self, text="Open Notebook", command=self.open_notebook).pack(side=tk.TOP, fill=tk.X)
            # Frame the notes within the grid
            self.notes_frame = tk.Frame(self)
            self.notes_frame.pack(fill=tk.BOTH, expand=True)
   
    # Initializing the noteform in order to access the note      
    def new_note(self):
        NoteForm(self)
    
    # Creating the adding note functionality as well as refreshing the display
    def add_note(self, note):
        self.notes.append(note)
        self.display_notes()
    
    # Displaying Note functionality 
    def display_notes(self):
        # Clear existing notes from previous usage
        for widget in self.notes_frame.winfo_children():
            widget.destroy()
        # Create a new button for each note
        for note in self.notes:
            btn = tk.Button(self.notes_frame, text=note.title, command=lambda n=note: self.show_note_content(n))
            btn.pack(side=tk.TOP, fill=tk.X)

    # Show a messagebox with the content of the note when its button is clicked.
    def show_note_content(self, note):
        messagebox.showinfo(note.title, f"Text: {note.text}\nLink: {note.link}\nTags: {', '.join(note.tags)}\nCreated At: {note.created_at}")
    
    # Save all notes to a JSON file
    def save_notebook(self):
        notes_dict = [note.to_dict() for note in self.notes]
        filename = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON files", "*.json")])
        if filename:
            with open(filename, "w") as file:
                json.dump(notes_dict, file)
    # Load notes from a JSON file and display them.
    def open_notebook(self):
        filename = filedialog.askopenfilename(filetypes=[("JSON files", "*.json")])
        if filename:
            # Converting dictionary objects into MakeNotes
            with open(filename, "r") as file:
                notes_dict = json.load(file)
            self.notes = [MakeNote(n["title"], n["text"], n["link"], n["tags"]) for n in notes_dict]
            self.display_notes()


# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example
# the submit method calls the MakeNote class that transforms the the entered data into a new note object.

class NoteForm(tk.Toplevel):
    # Initialize the note creation form as a top-level window.
    def __init__(self, master):
        super().__init__(master)
        self.title("New Note")
        self.geometry("400x300")
        self.master = master

        # Labels for the note object when writing
        tk.Label(self, text="Title").pack()
        self.title_entry = tk.Entry(self)
        self.title_entry.pack()

        tk.Label(self, text="Text").pack()
        self.text_entry = tk.Text(self, height=5)
        self.text_entry.pack()

        tk.Label(self, text="Link").pack()
        self.link_entry = tk.Entry(self)
        self.link_entry.pack()

        tk.Label(self, text="Tags (comma-separated)").pack()
        self.tags_entry = tk.Entry(self)
        self.tags_entry.pack()

        # Button for the submit note function
        tk.Button(self, text="Submit", command=self.submit).pack()

    # Collect data from the form, create a new note, and add it to the main window.
    def submit(self):
        # Retrieve data from input fields
        title = self.title_entry.get()
        text = self.text_entry.get("1.0", tk.END).strip()
        link = self.link_entry.get()
        tags = self.tags_entry.get().split(",")

        #Create a new MakeNote object and add it to the main window's notes list
        note = MakeNote(title, text, link, tags)
        self.master.add_note(note)
        self.destroy()

    
# The MakeNote class takes a dictionary containing the data entered into the form window,
# and transforms it into a new note object.
# At present the note objects have attributes but no methods.

class MakeNote():
    # Initialize a new note object with the given details.
    def __init__(self, title, text, link, tags):
        self.title = title
        self.text = text
        self.link = link
        self.tags = tags
        self.created_at = datetime.datetime.now().isoformat()
        
    # Helper function for dictionary conversion to note
    def to_dict(self):
        return {
            "title": self.title,
            "text": self.text,
            "link": self.link,
            "tags": self.tags,
            "created_at": self.created_at
        }
    
# main execution

if __name__ == '__main__':
    
    main_window = MainWindow() # this creates a notebook / main window called main_window. You may change the name if you want

    main_window.mainloop()

#### Print your three notes below

In [4]:
# 1. In order to have any application be able to access files within your 
# native device within the scope of this course, you have to import filedialog from tkinter
# 2. Utilizing the JSON API to data scrape, we can use it for applications such as entering
# location names and it will return the metadata
# 3. In order to be able to handle different file types such as JSON, txt, or csv
# you have to implement within your code defaultfiletypes and filetypes of the file
# you want to be able to handle